In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from llama_index import SimpleDirectoryReader, ServiceContext, LLMPredictor
from llama_index import GPTVectorStoreIndex, GPTListIndex, GPTSimpleKeywordTableIndex
from llama_index.composability import ComposableGraph
from langchain.chat_models import ChatOpenAI

#### Load Documents

In [5]:
reader = SimpleDirectoryReader('../paul_graham_essay/data')
documents = reader.load_data()

#### Parse into Nodes

In [6]:
from llama_index.node_parser import SimpleNodeParser
nodes = SimpleNodeParser().get_nodes_from_documents(documents)

#### Add to Docstore

In [7]:
from llama_index.storage.docstore import SimpleDocumentStore
docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

#### Define Multiple Indexes

Each index uses the same underlying Node.

In [ ]:
from llama_index.storage.storage_context import StorageContext


storage_context = StorageContext.from_defaults(docstore=docstore)
list_index = GPTListIndex(nodes, storage_context=storage_context)
vector_index = GPTVectorStoreIndex(nodes, storage_context=storage_context) 
keyword_table_index = GPTSimpleKeywordTableIndex(nodes, storage_context=storage_context) 

In [15]:
# NOTE: the docstore sitll has the same nodes
len(storage_context.docstore.docs)

6

#### Test out some Queries

In [11]:
llm_predictor_chatgpt = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context_chatgpt = ServiceContext.from_defaults(llm_predictor=llm_predictor_chatgpt, chunk_size=1024)

Unknown max input size for gpt-3.5-turbo, using defaults.


In [ ]:
query_engine = list_index.as_query_engine()
response = query_engine.query("What is a summary of this document?") 

In [ ]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("What did the author do growing up?") 

In [ ]:
query_engine = keyword_table_index.as_query_engine()
response = query_engine.query("What did the author do after his time at YC?")

In [13]:
print(response)



After his time at YC, the author decided to take a break and focus on painting. He spent most of 2014 painting and then, in November, he ran out of steam and stopped. He then moved to Florence, Italy to attend the Accademia di Belle Arti di Firenze, where he studied painting and drawing. He also started painting still lives in his bedroom at night. In March 2015, he started working on Lisp again and wrote a new Lisp, called Bel, in itself in Arc. He wrote essays through 2020, but also started to think about other things he could work on. He wrote an essay for himself to answer the question of how he should choose what to do next and then wrote a more detailed version for others to read. He also created the Y Combinator logo, which was an inside joke referencing the Viaweb logo, a white V on a red circle, so he made the YC logo a white Y on an orange square. He also created a fund for YC for a couple of years, but after Heroku got bought, he had enough money to go back to being self-f